In [1]:
import tensorflow as tf
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

### Cargamos el dataset usando Keras

In [2]:
fashion_mnist =tf.keras.datasets.fashion_mnist.load_data()

(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist

X_train, X_valid = X_train_full[:-5000], X_train_full[-5000:]
y_train, y_valid = y_train_full[:-5000], y_train_full[-5000:]

print("Dimensión de los datos de entrenamiento: ", X_train.shape)
print("Dimensión de los datos de validación: ", X_valid.shape)
print("Dimensión de los datos de prueba: ", X_test.shape)

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 1us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Dimensión de los datos de entrenamiento:  (55000, 28, 28)
Dimensión de los datos de validación:  (5000, 28, 28)
Dimensión de los datos de prueba:  (10000, 28, 28)


In [3]:
X_train, X_valid, X_test = X_train / 255., X_valid / 255., X_test / 255.

In [4]:
from pathlib import Path
import os
import datetime

run_logdir= os.path.join("fashion_mnist_cnn", datetime.datetime.now().strftime("%Y%m%d%H%M%S") )
run_logdir

'fashion_mnist_cnn\\20240921183522'

In [5]:
tensorboard_cb = tf.keras.callbacks.TensorBoard(run_logdir,
                                                histogram_freq = 1,
                                                profile_batch=(500, 520))

In [6]:
from functools import partial
tf.keras.backend.clear_session()

DefaultConv2D = partial(tf.keras.layers.Conv2D, kernel_size=3, padding='same', activation='relu', kernel_initializer='he_normal')

model = tf.keras.models.Sequential([
#64 filtros de 7x7, stride por defecto 1
    DefaultConv2D(filters=64, kernel_size=7, input_shape=[28, 28, 1]),
#capa de max pooling tamaño 2
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    DefaultConv2D(filters=128),
    DefaultConv2D(filters=128),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    DefaultConv2D(filters=256),
    DefaultConv2D(filters=256),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=128, activation="relu",
                          kernel_initializer="he_normal"),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(units=64, activation="relu",
                          kernel_initializer="he_normal"),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(units=10, activation="softmax")

])

c:\Users\yeyer\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [7]:
model.summary()
# {7x7}x 64 +64(1bias) = 3200
# {3x3}x 128x64 + 128 = 73856
# {3x3}x 128x128 + 128 = 147584

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 28, 28, 64)     │         3,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 14, 14, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 14, 14, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 7, 7, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 7, 7, 256)      │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 7, 7, 256)      │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 3, 3, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 2304)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       295,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,413,834 (5.39 MB)

 Trainable params: 1,413,834 (5.39 MB)

 Non-trainable params: 0 (0.00 B)

Notar que después de la primera capa de Max pooling se repite la misma estructura 2 veces: 2 capas de convolución seguidas de una capa Max pooling. Para imágenes más grandes, se puede repetir la estructura más veces. El número de repeticiones es un hiperparámetro.

Es una práctica común duplicar el número de filtros después de cada capa de pooling (AlexNet). Dado que una capa de pooling divide cada dimensión espacial por un factor de 2, podemos duplicar el número de mapas de características en la siguiente capa sin temor a incrementar demasiado la cantidad de parámetros, uso de memoria o carga computacional.

In [8]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid), callbacks=[tensorboard_cb])

Epoch 1/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 171s 97ms/step - accuracy: 0.6202 - loss: 1.0673 - val_accuracy: 0.8638 - val_loss: 0.3768
Epoch 2/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 149s 87ms/step - accuracy: 0.8548 - loss: 0.4307 - val_accuracy: 0.8802 - val_loss: 0.3079
Epoch 3/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 141s 82ms/step - accuracy: 0.8852 - loss: 0.3408 - val_accuracy: 0.8972 - val_loss: 0.2867
Epoch 4/10
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 159s 92ms/step - accuracy: 0.8967 - loss: 0.3090 - val_accuracy: 0.9080 - val_loss: 0.2732
Epoch 5/10
 982/1719 ━━━━━━━━━━━━━━━━━━━━ 1:07 92ms/step - accuracy: 0.9047 - loss: 0.2821

KeyboardInterrupt: 

In [13]:
# model.save("./redes_CNN/cnn_mnist_original.h5")

d:\codigos_de_clase\.venv\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [13]:
model1 = tf.keras.models.load_model("./redes_CNN/cnn_mnist_original.h5")

c:\Users\yeyer\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [14]:
class_names = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat", "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]
X_nuevo = X_test[:10]
y_pred = np.argmax(model1.predict(X_nuevo), axis=-1)
np.array(class_names)[y_pred]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 226ms/step


array(['Ankle boot', 'Pullover', 'Trouser', 'Trouser', 'Shirt', 'Trouser',
       'Coat', 'Shirt', 'Sandal', 'Sneaker'], dtype='<U11')

In [15]:
salidas = y_test[:10]
np.array(class_names)[salidas]

array(['Ankle boot', 'Pullover', 'Trouser', 'Trouser', 'Shirt', 'Trouser',
       'Coat', 'Shirt', 'Sandal', 'Sneaker'], dtype='<U11')

In [18]:
model1.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])
score = model1.evaluate(X_test, y_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 8s 24ms/step - accuracy: 0.9099 - loss: 0.3149
